In [71]:
import pandas as pd, numpy as np
import json
import nltk 
import re
import collections, itertools
from IPython.display import clear_output
import time
from scipy.sparse import csr_matrix

In [2]:
stop = nltk.corpus.stopwords.words('english')

In [3]:
datafile =  'assignment3_aricles.json'

In [4]:
articles = pd.read_json(datafile, orient='records', encoding="utf-8")

In [5]:
articles.head()

,Content,Title,article_id
0,Tikcro enters into research and license agreem...,Tikcro enters into research and license agreem...,0
1,A North Carolina woman is trying to warn other...,Facebook Friend Request Nearly Cost One North ...,1
2,LONDON--(BUSINESS WIRE)--\n\nAMLIN plc\n\nTOTA...,Amlin plc UK Regulatory Announcement: Total Vo...,2
3,Khaleda asks for security\n\n\n\nBNP Chairpers...,Khaleda asks for security,3
4,Liberian Health Clinics Reopen Slowly with Ren...,Liberian Health Clinics Reopen Slowly with Ren...,4


In [7]:
articles['Content'] = articles['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
articles['Title'] = articles['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
# articles.Content[0]

In [9]:
#You can use n-gram at word level for this task
#try with different n-gram values 
# You can use ngrams from nltk for this

def getNgrams(articles):
    ngrams_list = []
    for i,row in articles.iterrows():
#     articles = articles.lower()
#     articles = re.sub(r'[^a-zA-Z0-9\s]', ' ', str(articles))
   
        art_content= re.sub(r'[^A-Za-z0-9]\\b', ' ', row['Content']).lower()
        art_content= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', art_content)
        art_title= re.sub(r'[^A-Za-z0-9]\\b', ' ', row['Title']).lower()
        art_title= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', art_title)
     
  
        tokens_content = [token for token in art_content.split(" ") if token != ""]
        output_content = list(nltk.ngrams(tokens_content, 2))
        tokens_title = [token for token in art_title.split(" ") if token != ""]
        output_title = list(nltk.ngrams(tokens_title, 2))
        total= list(set(output_content + output_title))
        ngrams_list.append(total)
    # Replaces escape character with space 
#     f = s.replace("\n", " ") 
    articles['ngrams'] = ngrams_list
    return articles
  

    #return articles with a field ngrams

In [10]:
articles =getNgrams(articles)

In [11]:
shingles =[]
for i in articles.ngrams:
    shingles.extend(i)
counter= collections.Counter(shingles).most_common(10055)

In [12]:
# shingles = [' '.join(x) for x in rand]
c_shingles=counter[55:]

In [13]:
unique_shingles =[]
for i in c_shingles:
    unique_shingles.append(i[0])

In [ ]:
# unique_shingles = [' '.join(x) for x in unique_]
# unique_shingles

## Convert n-grams into binary vector representation for each document. You can do some optimzations if the matrix is too big

In [14]:
# def getBinaryMatrix(articles,shingles):
#     binary_matrix = csr_matrix((len(shingles),len(articles))dtype=np.int8).toarray()
   
#     found=0
#     for s in range(len(shingles)):
#         for j,row in articles.iterrows():
#             if shingles[s] in row['ngrams']:
#                 clear_output()
#                 found = found +1
# #                 print ('Found ', 1)
#                 binary_matrix[s,j] = 1
#                 print(found)
#     return binary_matrix


In [ ]:
def getBinaryMatrixnp(articles, shingles):
    bin_matrix = pd.DataFrame(csr_matrix((len(shingles),len(articles))))

    for i, gram in enumerate(shingles):
        t = articles[articles['ngrams'].apply(lambda x: gram in x)]
        print(len(t))
        bin_matrix.loc[i, list(t['article_id'])] = 1
        clear_output()
    return bin_matrix

In [ ]:
binary_matrix = getBinaryMatrixnp(articles,unique_shingles)

In [18]:
np.sum(np.sum(binary_matrix))

1705716.0

In [ ]:

# j.sort_index()

## We need hash function that maps integers 0, 1, . . . , k − 1 to bucket numbers 0 through k − 1. It might be impossible to avoid collisions but as long as the collions are too many it won't matter much.

* The simplest would be using the builtin hash() function, it can be for example, hash(rownumber) % Numberofbuckets
* You can generate several of these hash functions by xoring a random integer (hash(rownumber)^randint) % Numberofbuckets
* It can also be a as simple as (rownumber * randint) % Numberofbuckets

In [58]:
def getHashFunctionValues(numrows, numhashfunctions):
    hash_mat = np.zeros((numrows,numhashfunctions))
    rowindex=np.array([i for i in range(numrows)])
    for i in range(numrows):
        np.random.shuffle(rowindex)
        for j in range(len(numhashfunctions)):
            hash_mat[i,j]=(hash(rowindex[j])%numrows)
    return hash_mat
    #return a matrix with hash values

In [20]:
hash_val_matrix=getHashFunctionValues(10000,100)

In [60]:
# hash_val_matrix=pd.DataFrame(hash_val)

In [73]:
# for i in hash_val_matrix:


## Compute minhash following the faster algorithm from the lecture 

In [ ]:
def getMinHashsig(binary_matrix, hash_val_matrix):
    signature_matrix=np.full((hash_val_matrix.shape[0],binary_matrix.shape[1]),np.inf)
    binary_matrix = binary_matrix.as_matrix()
    rows, cols = np.where(binary_matrix == 1)
    
    for j in range(len(rows)):
        r,c = rows[j], cols[j]
        for i in range(len(hash_val_matrix)):
            if hash_val_matrix[i,r] < signature_matrix[i,c]:
                signature_matrix[i,c] = hash_val_matrix[i,r]
                
    return signature_matrix

In [22]:
signature_matrix = getMinHashsig(binary_matrix, hash_val_matrix)

9999 48183


## Hash signature bands into buckets. Find a way to combine all the signature values in a band and hash them into a number of buckets ususally very high.
* Easiest way is to add all the signature values in the bucket and use a similar hash function like before

In [ ]:
def getLSH(signature_matrix, hashfunctions, num_bands, num_buckets):
    buckets={}
    for i in range(len(bands)):
#     print(bands[i])
        for j in bands[i].T:
            m=str(j)
            l=hash(m)
            if l not in buckets:
                buckets[l]=list(j)
            
            else:
                buckets[l].append(j)
#             buckets.setdefault('l', []).append(list[j])
    return buckets
#return lsh buckets or hash table

In [ ]:
LSH = getLSH(signature_matrix,20)

In [ ]:
def find_candidate_pairs(LSH):
    candidate_pairs = []
    count=0
    for key, val in LSH.items():
        clear_output()
        count=count+1
        if len(val) > 1:
            val=list(set(val))           
            
            pairs =list(itertools.combinations(val,2))
            candidate_pairs.extend(pairs)
        
    return cantidate_pairs

In [ ]:
candidates = find_candidate_pairs(LSH)

In [ ]:
def jaccard_similarity(candidate_1, candidate_2):
    intersect = len(list(set(candidate_1).intersect(candidate_2)))
    union = (len(candidate_1) + len(candidate_2)) - intersect
    j_sim= float(intersect / union)
    return j_sim

In [ ]:
def choose_candidates(candidate_pairs, threshold):
    candidates = []

    for pair in candidate_pairs:
        candidate_1 = signature_matrix[:, pair[0]]
        candidate_2 = signature_matrix[:, pair[1]]
        
        if(jaccard_similarity(candidate_1, candidate_2) >= threshold):
            candidates.append(pair)
            
    return candidates

In [ ]:
candidates = choose_candidates(candidates, threshold = 0.7)

## Tune parameters to make sure the threshold is appropriate.
## plot the probability of two similar items falling in same bucked for different threshold values


In [47]:
candidates 

[(6851, 4),
 (6851, 9284),
 (6851, 38534),
 (6851, 36264),
 (6851, 44552),
 (6851, 37804),
 (6851, 13457),
 (6851, 10196),
 (6851, 14421),
 (6851, 41239),
 (6851, 25049),
 (6851, 30591),
 (4, 9284),
 (4, 38534),
 (4, 36264),
 (4, 44552),
 (4, 37804),
 (4, 13457),
 (4, 10196),
 (4, 14421),
 (4, 41239),
 (4, 25049),
 (4, 30591),
 (9284, 38534),
 (9284, 36264),
 (9284, 44552),
 (9284, 37804),
 (9284, 13457),
 (9284, 10196),
 (9284, 14421),
 (9284, 41239),
 (9284, 25049),
 (9284, 30591),
 (38534, 36264),
 (38534, 44552),
 (38534, 37804),
 (38534, 13457),
 (38534, 10196),
 (38534, 14421),
 (38534, 41239),
 (38534, 25049),
 (38534, 30591),
 (36264, 44552),
 (36264, 37804),
 (36264, 13457),
 (36264, 10196),
 (36264, 14421),
 (36264, 41239),
 (36264, 25049),
 (36264, 30591),
 (44552, 37804),
 (44552, 13457),
 (44552, 10196),
 (44552, 14421),
 (44552, 41239),
 (44552, 25049),
 (44552, 30591),
 (37804, 13457),
 (37804, 10196),
 (37804, 14421),
 (37804, 41239),
 (37804, 25049),
 (37804, 30591),
 

In [ ]:
cantidates_df = pd.DataFrame()
x,y= [],[]
for i in range(len(candidates)):
    x.append(candidates[i][0])
    y.append(candidates[i][1])
cantidates_df['doc'] = x
cantidates_df['pair'] = y

## Choose the best parameters and get nearest neighbors of each articles

In [ ]:
cantidates_df.to_csv('submissions.csv',sep=',')

## Write the results to submissions.csv file and get the score